# Dado el dataset de rating de goodreads (1.1GB) https://www.kaggle.com/bahramjannesarr/goodreads-book-datasets-10m determinar:

### 1.- Rating promedio de todos los libros

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark = SparkSession\
    .builder\
    .appName("GoodReads")\
    .getOrCreate()

In [98]:
# Schema que finalmente no he usado

schema = StructType([ \
                     StructField("Id",IntegerType(),False),\
                     StructField("Name",StringType(),False),\
                     StructField("RatingDist1",StringType(),False),\
                     StructField("pagesNumber",IntegerType(),False),\
                     StructField("RatingDist4",StringType(),False),\
                     StructField("RatingDistTotal",StringType(),False),\
                     StructField("PublishMonth",IntegerType(),False),\
                     StructField("PublishDay",IntegerType(),False),\
                     StructField("Publisher",StringType(),False),\
                     StructField("CountsOfReview",IntegerType(),False),\
                     StructField("PublishYear",IntegerType(),False),\
                     StructField("Language",StringType(),False),\
                     StructField("Authors",StringType(),False),\
                     StructField("Rating",StringType(),False),\
                     StructField("RatingDist2",StringType(),False),\
                     StructField("RatingDist5",StringType(),False),\
                     StructField("ISBN",StringType(),False),\
                     StructField("RatingDist3",StringType(),False)])

# Mi método (SALE MAL)

df=spark.createDataFrame([],StructType([]))

file_lst=['books'] 

for i in file_lst:
    tmp_df = spark.read.format("csv")\
    .option("header","true")\
    .option("delimiter",",")\
    .option("inferSchema", "true")\
    .option("quote", "\"")\
    .option("escape","\"")\
    .option("multiLine","true")\
    .load(i)
    df1=df.unionByName(tmp_df,allowMissingColumns=True)
    df=df1

df.select("PublishDay").distinct().show(10,False)

In [104]:
df.limit(10).toPandas()

,Id,Name,RatingDist1,pagesNumber,RatingDist4,RatingDistTotal,PublishMonth,PublishDay,Publisher,CountsOfReview,PublishYear,Language,Authors,Rating,RatingDist2,RatingDist5,ISBN,RatingDist3
0,2000004,Geschichte der Prophetie in Israel : von den A...,Joseph Blenkinsopp,3170117742,3.66,1998,8,1,Kohlhammer,5:10,4:24,3:25,2:3,1:0,total:62,0,None,296
1,2000006,Josué et l'alliance de Sichem : (Josué 24:1-28),Moshé Anbar,3631450184,0.0,1992,6,1,"Peter Lang Gmbh, Internationaler Verlag Der Wi...",5:0,4:0,3:0,2:0,1:0,total:0,0,None,163
2,2000012,The Essential Chocolate Chip Cookbook: Recipes...,Elinor Klivans,0811858049,3.55,2007,12,13,Chronicle Books,5:9,4:12,3:10,2:5,1:2,total:38,6,None,120
3,2000021,Fifty Fifty Tutti Frutti Chocolate Chip And Ot...,Esther Menon,0435125370,3.0,2009,9,1,Heinemann Educational Secondary Division,5:0,4:1,3:7,2:1,1:0,total:9,0,None,199
4,2000024,Let's Make Cookies,Mary Hill,0516239589,3.5,2005,12,2,Children's Press(CT),5:0,4:1,3:1,2:0,1:0,total:2,0,None,24
5,2000026,The Case for Penal Abolition,W. Gordon West,1551301474,4.33,2000,5,1,Canadian Scholars Press,5:1,4:2,3:0,2:0,1:0,total:3,1,None,362
6,2000030,"Cracking the GRE Biology, 3rd Edition (Crackin...",Deborah Guest,0375756159,0.0,2002,9,17,Princeton Review,5:0,4:0,3:0,2:0,1:0,total:0,0,None,352
7,2000031,Daring to Draw Near: People in Prayer,John White,0877847886,4.18,1977,10,1,IVP Books,5:99,4:66,3:33,2:8,1:4,total:210,17,None,162
8,2000032,Blondels Song: The Capture Imprisonment And Ra...,David Boyle,067091486X,3.87,2005,7,26,Viking,5:27,4:48,3:31,2:5,1:0,total:111,2,None,320
9,2000034,"Richard Lionheart, the Crusader King",John Matthews,1853140074,2.67,1988,8,31,Firebird Books Ltd.,5:0,4:0,3:2,2:1,1:0,total:3,0,None,52


In [78]:
# Método de sara (funciona bien)

booksDF = spark.read.format("csv")\
.option("header","true")\
.option("delimiter",",")\
.option("inferSchema","true")\
.option("quote","\"")\
.option("escape","\"")\
.option("multiLine","true")\
.load("books/book1-100k.csv")

di = [100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,3000,4000,5000]

for i in range(len(di)-1):
    directorio_books = "C:/Users/rafael.gomez/Documents/EjerciciosSpark/Goodreads Dataset Project/books/book"+str(di[i])+"k-"+str(di[i+1])+"k.csv"
    booksDF_bis = spark.read.format("csv")\
    .option("header","true")\
    .option("delimiter",",")\
    .option("inferSchema","true")\
    .option("quote","\"")\
    .option("escape","\"")\
    .option("multiLine","true")\
    .load(directorio_books)
    
    booksDF1 = booksDF.unionByName(booksDF_bis, allowMissingColumns=True)
    booksDF = booksDF1

booksDF.select("PublishDay").distinct().show(5,False)

+----------+
|PublishDay|
+----------+
|31        |
|28        |
|26        |
|27        |
|12        |
+----------+
only showing top 5 rows



In [101]:
booksDF.limit(10).toPandas()

,Id,Name,RatingDist1,pagesNumber,RatingDist4,RatingDistTotal,PublishMonth,PublishDay,Publisher,CountsOfReview,PublishYear,Language,Authors,Rating,RatingDist2,RatingDist5,ISBN,RatingDist3,Description,Count of text reviews
0,1,Harry Potter and the Half-Blood Prince (Harry ...,1:9896,652,4:556485,total:2298124,16,9,Scholastic Inc.,28062,2006,eng,J.K. Rowling,4.57,2:25317,5:1546466,None,3:159960,None,NaN
1,2,Harry Potter and the Order of the Phoenix (Har...,1:12455,870,4:604283,total:2358637,1,9,Scholastic Inc.,29770,2004,eng,J.K. Rowling,4.50,2:37005,5:1493113,0439358078,3:211781,None,NaN
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,1:108202,309,4:1513191,total:6587388,1,11,Scholastic Inc,75911,2003,eng,J.K. Rowling,4.47,2:130310,5:4268227,None,3:567458,None,NaN
3,4,Harry Potter and the Chamber of Secrets (Harry...,1:11896,352,4:706082,total:2560657,1,11,Scholastic,244,2003,eng,J.K. Rowling,4.42,2:49353,5:1504505,0439554896,3:288821,None,NaN
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,1:10128,435,4:630534,total:2610317,1,5,Scholastic Inc.,37093,2004,eng,J.K. Rowling,4.57,2:24849,5:1749958,043965548X,3:194848,None,NaN
5,6,Harry Potter and the Goblet of Fire (Harry Pot...,1:9419,734,4:606800,total:2431085,28,9,Scholastic,31978,2002,eng,J.K. Rowling,4.56,2:24282,5:1612165,None,3:178419,None,NaN
6,8,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...",1:402,2690,4:4650,total:43968,13,9,Scholastic,166,2004,eng,J.K. Rowling,4.78,2:283,5:37432,0439682584,3:1201,None,NaN
7,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",1:0,152,4:7,total:28,26,4,Nimble Books,1,2005,en-US,W. Frederick Zimmerman,3.79,2:5,5:10,0976540606,3:6,None,NaN
8,10,"Harry Potter Collection (Harry Potter, #1-6)",1:257,3342,4:4358,total:30313,12,9,Scholastic,809,2005,eng,J.K. Rowling,4.73,2:218,5:24406,0439827604,3:1074,None,NaN
9,12,The Ultimate Hitchhiker's Guide: Five Complete...,1:3443,815,4:75683,total:274268,1,11,Gramercy Books,255,2005,eng,Douglas Adams,4.37,2:7613,5:157499,0517226952,3:30030,None,NaN


In [82]:
# Calcular rating promedio de todos los libros (lo que se me pedía, vaya)

booksDF.select(F.avg(F.col("Rating"))).show()

+-----------------+
|      avg(Rating)|
+-----------------+
|2.894242694467414|
+-----------------+



### 2.- Rating promedio de los libros por autor

In [91]:
# No me pide que lo ordene pero por qué no
booksDF.groupBy(F.col("Authors")).agg(F.mean(F.col("Rating"))).orderBy(F.col("avg(Rating)").desc()).show(10,False)

+---------------------+-----------+
|Authors              |avg(Rating)|
+---------------------+-----------+
|Men-Tal              |5.0        |
|Bruce R. Archambeault|5.0        |
|Donald MacKechnie    |5.0        |
|Morton Rosoff        |5.0        |
|Ranen Omer-Sherman   |5.0        |
|G. Srinivasan        |5.0        |
|BRANDIMONT           |5.0        |
|Fatemeh Ebtehaj      |5.0        |
|William Whallon      |5.0        |
|Elizabeth  Stewart   |5.0        |
+---------------------+-----------+
only showing top 10 rows



### 3.- Rating promedio de los libros por Publisher

In [92]:
# No me pide que lo ordene pero por qué no
booksDF.groupBy(F.col("Publisher")).agg(F.mean(F.col("Rating"))).orderBy(F.col("avg(Rating)").desc()).show(10,False)

+-------------------------------------------------+-----------+
|Publisher                                        |avg(Rating)|
+-------------------------------------------------+-----------+
|David Daniluk                                    |5.0        |
|Susan Chapman Melanson                           |5.0        |
|Indigo Reef Publishing Inc                       |5.0        |
|Mountain Cat Productions                         |5.0        |
|National Assn of Office and Industrial Properties|5.0        |
|Seven Seas Publishing Pty                        |5.0        |
|Writeart                                         |5.0        |
|L.T.P. Publications                              |5.0        |
|Browngrotta Arts                                 |5.0        |
|Rock Reef Publishing House                       |5.0        |
+-------------------------------------------------+-----------+
only showing top 10 rows



### 4.- Número promedio de páginas de todos los libros

In [95]:
booksDF.select(F.avg(F.col("pagesNumber"))).show()

+------------------+
|  avg(pagesNumber)|
+------------------+
|276.55165080445977|
+------------------+



### 5.- Número promedio de páginas de todos los libros por autor

In [96]:
# No me pide que lo ordene pero por qué no
booksDF.groupBy(F.col("Authors")).agg(F.mean(F.col("pagesNumber"))).orderBy(F.col("avg(pagesNumber)").desc()).show(10,False)

+------------------------------------+------------------+
|Authors                             |avg(pagesNumber)  |
+------------------------------------+------------------+
|Sandy Redburn                       |1807321.6         |
|A.B. Murphy                         |751507.3333333334 |
|John B. Hare                        |500000.0          |
|Logos Research Systems              |100000.0          |
|Progressive Management              |35428.4375        |
|Timothy McVeigh                     |33133.0           |
|Robert H. Wozniak                   |22100.0           |
|Veterans Affairs Department Research|16153.0           |
|World Spaceflight News              |13942.333333333334|
|Keith Crook                         |9999.0            |
+------------------------------------+------------------+
only showing top 10 rows



### 6.- Número promedio de páginas de todos los libros por Publisher

In [97]:
# No me pide que lo ordene pero por qué no
booksDF.groupBy(F.col("Publisher")).agg(F.mean(F.col("pagesNumber"))).orderBy(F.col("avg(pagesNumber)").desc()).show(10,False)

+-----------------------------------------------------------+------------------+
|Publisher                                                  |avg(pagesNumber)  |
+-----------------------------------------------------------+------------------+
|Crafty Secrets Publications                                |1807321.6         |
|Sacred-texts.com                                           |500000.0          |
|Department of Russian Language and Literature University of|322128.5714285714 |
|Logos Research Systems                                     |100000.0          |
|Encyclopedia Britannica, Incorporated                      |32642.0           |
|Progressive Management                                     |19106.3625        |
|Still Waters Revival Books                                 |10080.142857142857|
|P. Shalom Publications, Incorporated                       |8539.0            |
|Hendrickson Publishers, Inc. (Peabody, MA)                 |6448.0            |
|IEEE/EMB                   

### 7.- Número promedio de libros publicados por autor

In [113]:
booksDF.groupBy(F.col("Authors")).agg(F.count("*").alias("conteoLibros")).agg(F.mean(F.col("conteoLibros")).alias("PromedioAutores")).show()

+------------------+
|   PromedioAutores|
+------------------+
|2.7400268625729134|
+------------------+



### 8.- Ordenar los libros de mayor a menor (Top 15) por número de ratings dados por usuarios (excluir aquellos valores sin rating)

In [127]:
booksDF.where(F.col("Rating") != 0).orderBy(F.col("CountsOfReview").desc()).select(F.col("Id"),F.col("Name"),F.col("Rating"),F.col("CountsOfReview"))\
.show(15,False)

+-------+---------------------------------------------------------+------+--------------+
|Id     |Name                                                     |Rating|CountsOfReview|
+-------+---------------------------------------------------------+------+--------------+
|2767052|The Hunger Games (The Hunger Games, #1)                  |4.33  |154447        |
|41865  |Twilight (Twilight, #1)                                  |3.59  |94850         |
|19063  |The Book Thief                                           |4.37  |87685         |
|4667024|The Help                                                 |4.46  |76040         |
|3      |Harry Potter and the Sorcerer's Stone (Harry Potter, #1) |4.47  |75911         |
|3636   |The Giver (The Giver, #1)                                |4.13  |57034         |
|43641  |Water for Elephants                                      |4.09  |52918         |
|2429135|The Girl with the Dragon Tattoo (Millennium, #1)         |4.14  |52225         |
|136251 |H

### 9.- Obtener Top 5 de ratings más frecuentes otorgados por usuarios

In [128]:
booksDF.groupBy(F.col("Rating")).agg(F.count("*").alias("Conteo")).orderBy(F.col("Conteo").desc()).show(5)

+------+------+
|Rating|Conteo|
+------+------+
|   0.0|451783|
|   4.0|151979|
|   3.0| 87288|
|   5.0| 79827|
|   3.5| 45222|
+------+------+
only showing top 5 rows

